In [20]:
%pip install numpy matplotlib scipy pandas seaborn scikit-learn tensorflow sklearn transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
    

In [21]:
## Imports
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score

from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding, AutoModelForSequenceClassification, TrainingArguments, Trainer


In [22]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # Convert logits to probabilities using softmax
    probs = np.exp(logits) / np.sum(np.exp(logits), axis=-1, keepdims=True)
    
    # Get predictions by selecting the class with the highest probability
    preds = np.argmax(probs, axis=-1)

    # Compute AUC for the positive class
    auc = roc_auc_score(labels, probs[:, 1], multi_class='ovr')

    # Compute accuracy
    accuracy = accuracy_score(labels, preds)
    return {"roc_auc": auc, "accuracy": accuracy}

def preprocess_function(examples):
    return tokenizer(examples["text"], max_length=128, padding=True, truncation=True)

In [23]:
path_ds = r"/kaggle/input/dataset-final/final_dataset_v1_afternb1.csv"
df_ds = pd.read_csv(path_ds)

# First, split into train and test sets (80% train, 20% test)
train_essays, test_essays = train_test_split(df_ds, test_size=0.2, random_state=42)

# Then, split the train set into train and validation sets (67% train, 33% validation)
train_essays, val_essays = train_test_split(train_essays, test_size=0.33, random_state=42)

# Check the size of each set
print(f'Training set size: {len(train_essays)}')
print(f'Validation set size: {len(val_essays)}')
print(f'Test set size: {len(test_essays)}')

# Show first few rows of the train set
train_essays.head()

Training set size: 38084
Validation set size: 18759
Test set size: 14211


,text,label,source,word_length,text_no_sw
39759,self-reliance is a concept that has become inc...,1,mistral7binstruct_v1,376,self-reliance concept become increasingly impo...
35109,marijuana is a drug that can be made from a pl...,1,falcon_180b_v1,269,marijuana drug made plant called cannabis . us...
24262,"today, the majority of humans own and operate ...",0,persuade_corpus,542,"today , majority humans operate cell phones da..."
60113,it is century people have their own opinion a...,0,personal-sunilthite,439,century people opinion decisions . human revo...
7659,"with the information the article ""making mona ...",0,persuade_corpus,656,information article `` making mona lisa smile ...


In [26]:
# Download the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

## Dataset
train_essay_dataset = Dataset.from_pandas(train_essays)
val_essay_dataset = Dataset.from_pandas(val_essays)
test_essay_dataset = Dataset.from_pandas(test_essays)

## Tokenize datasets
tokenized_train_essays = train_essay_dataset.map(preprocess_function, batched=True)
tokenized_val_essays = val_essay_dataset.map(preprocess_function, batched=True)

## Training
training_args = TrainingArguments(
    output_dir="/kaggle/working/",
    learning_rate=2e-5,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_essays,
    eval_dataset=tokenized_val_essays,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/38084 [00:00<?, ? examples/s]

Map:   0%|          | 0/18759 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Roc Auc,Accuracy
1,0.041600,0.048411,0.999353,0.988059
2,0.016900,0.046461,0.999541,0.991151


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=4762, training_loss=0.04247177102795072, metrics={'train_runtime': 883.623, 'train_samples_per_second': 86.2, 'train_steps_per_second': 5.389, 'total_flos': 2522444205207552.0, 'train_loss': 0.04247177102795072, 'epoch': 2.0})

In [27]:
# # Tokenize the test dataset
# tokenized_test_essays = test_essay_dataset.map(preprocess_function, batched=True)

# # Evaluate the model on the test dataset
# test_results = trainer.evaluate(tokenized_test_essays)

# # Print the evaluation results with corrected keys
# print(f"Test AUC: {test_results['eval_roc_auc']:.4f}")
# print(f"Test Accuracy: {test_results['eval_accuracy']:.4f}")


Map:   0%|          | 0/14211 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Test AUC: 0.9994
Test Accuracy: 0.9901
